In [2]:
import pandas as pd
df = pd.read_csv('./supermarket_data.csv', parse_dates=['发货日期'])
df.loc[:,'发货年'] = df['发货日期'].dt.year.astype('str')
df.loc[:,'销售额'] = df['销售额'].str.replace(',','').astype('float')
df.loc[:,'利润率'] = df['利润率'].str.replace('%','').astype('float')
df.loc[:,'折扣'] = df['折扣'].str.replace('%','').astype('float')
df.head()

,利润率,制造商,产品名称,利润,发货日期,国家,地区,城市,子类别,客户名称,折扣,数量,省/自治区,类别,细分,订单 Id,订单日期,邮寄方式,销售额,发货年
0,-47.0,Fiskars,"Fiskars 剪刀, 蓝色",-61,2017-04-29,中国,华东,杭州,用品,曾惠,40.0,2,浙江,办公用品,公司,US-2017-1357144,2017/4/27,二级,130.0,2017
1,34.0,GlobeWeis,"GlobeWeis 搭扣信封, 红色",43,2017-06-19,中国,西南,内江,信封,许安,0.0,2,四川,办公用品,消费者,CN-2017-1973789,2017/6/15,标准级,125.0,2017
2,13.0,Cardinal,"Cardinal 孔加固材料, 回收",4,2017-06-19,中国,西南,内江,装订机,许安,40.0,2,四川,办公用品,消费者,CN-2017-1973789,2017/6/15,标准级,32.0,2017
3,-8.0,Kleencut,"Kleencut 开信刀, 工业",-27,2017-12-13,中国,华东,镇江,用品,宋良,40.0,4,江苏,办公用品,公司,US-2017-3017568,2017/12/9,标准级,321.0,2017
4,40.0,KitchenAid,"KitchenAid 搅拌机, 黑色",550,2016-06-02,中国,中南,汕头,器具,万兰,0.0,3,广东,办公用品,消费者,CN-2016-2975416,2016/5/31,二级,1376.0,2016


In [3]:
mapdata = df[['省/自治区','销售额']].groupby(['省/自治区'], as_index=False).sum()
mapdata

,省/自治区,销售额
0,上海,582447.0
1,云南,422326.0
2,内蒙古,273454.0
3,北京,409146.0
4,吉林,666785.0
5,四川,400881.0
6,天津,549912.0
7,宁夏,58122.0
8,安徽,563334.0
9,山东,1586781.0


In [4]:
#%%
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts import options as opts
from pyecharts.charts import Map

c = (
    Map(init_opts=opts.InitOpts('1000px','600px'))
    .add("各省销售额", mapdata.values.tolist(), "china")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各省销售数据汇总"),
        visualmap_opts=opts.VisualMapOpts(max_=1600000)
        )
)

c.render()
c.load_javascript()
#%%

In [5]:
c.render_notebook()

In [6]:
bardata = df[['类别','利润率','折扣']].groupby(['类别'],as_index=False).mean().round(decimals=2)
bardata

,类别,利润率,折扣
0,办公用品,8.70,8.56
1,家具,7.92,14.61
2,技术,8.82,12.47


In [7]:
from pyecharts.globals import CurrentConfig, NotebookType, ThemeType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts import options as opts
from pyecharts.charts import Bar


bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
      .add_xaxis(bardata['类别'].tolist())
      .add_yaxis("利润率", bardata['利润率'].tolist())
      .add_yaxis("折扣率", bardata['折扣'].tolist())
      .set_global_opts(
          title_opts={"text": "不同类别商品对比", "subtext": "利润率及折扣率均值（%）"}
          )
)

bar.render()
bar.load_javascript()
#%%

In [8]:
bar.render_notebook()

In [9]:
bar3d_data = df[['子类别','发货年','销售额']].groupby(['子类别','发货年'], as_index=False).sum()
bar3d_data = bar3d_data.loc[bar3d_data['发货年']!='2018']
bar3d_data.head()

,子类别,发货年,销售额
0,书架,2014,356760.0
1,书架,2015,529288.0
2,书架,2016,574102.0
3,书架,2017,826332.0
5,信封,2014,50640.0


In [12]:
#%%
from pyecharts.globals import CurrentConfig, NotebookType, ThemeType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts.charts import Bar3D
from pyecharts import options as opts
bar3d = (
    Bar3D()
    .add('销售额', 
        data=bar3d_data.values.tolist(),
        xaxis3d_opts=opts.Axis3DOpts(bar3d_data['子类别'].unique().tolist(),
        type_='category', interval=0, name='子类别'),
        yaxis3d_opts=opts.Axis3DOpts(bar3d_data['发货年'].unique().tolist(),
        type_='category', interval=0, name='发货年'),
        zaxis3d_opts=opts.Axis3DOpts(type_='value', interval=0, name='销售额'), 
    )
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(max_=830000)
    )
)
#%%

bar3d.load_javascript()


In [13]:
bar3d.render_notebook()